#  Comparison of CNN to Vision Transformer model trained on HiRISE Mars Satellite Images     (Feel free to change this)
### by Aniruddha Prasad and Andrew Hartnett

The following notebook will compare the accuracies of a Convolutional Neural Network (CNN) and Vision Transformer (ViT) trained on satellite images taken of Mars from the HiRISE dataset. The goal of this work is to determine whether or not a pre-trained ViT model, which has been seen used as the state-of-the-art for image classification in certain circumstances, will prove better when pre-trained on a significant size dataset and fine-tuned to this data. Then, we will train 3 version of each model with larger and larger subsets of the data to determine the trend in accuracy for each model. This will tell us which model will be best as more images are accumulated over the years.

### Table of Contents

1. Prepare the Training Data
2. Define and Train the CNN - **WIP**
3. Define and Train the Vision Transformer (ViT) - **WIP**
4. Evaluate CNN vs ViT - **WIP**
5. Retrain CNN and ViT on small, medium, and full HiRISE - **WIP**
6. Compare three CNNs vs three ViTs - **WIP**

## 1. Prepare the training data

In [2]:
# Tendorflow imports
import tensorflow as tf

# Helper libraries
import math
import numpy as np
import matplotlib
matplotlib.use('PS') #prevent import error due to venv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

# Imports for dataset separation
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Improve progress bar display
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm

# Import for comparison of CNN and VIT
from sklearn.metrics import classification_report

## Data Preprocessing

In [ ]:
data_images = []
data_labels = []
rel_img_path = 'hirise-map-proj-v3/map-proj/' # add path of folder to image name for later loading

# open up the labeled data file
with open('hirise-map-proj-v3/labels-map-proj.txt') as labels:
  for line in labels:
    file_name, label = line.split(' ')
    data_images.append(rel_img_path + file_name)
    data_labels.append(int(label))

# divide data into testing and training (total len 3820)
train_images, test_images, train_labels, test_labels = train_test_split(
    data_images, data_labels, test_size=0.15, random_state=666)
test_len = len(test_images)   # 573
train_len = len(train_images) # 3247

# label translations
class_labels = ['other','crater','dark_dune','streak',
                'bright_dune','impact','edge']


In [ ]:
# Print length of training set (should be 3247)
print(train_len)

In [ ]:
# Print length of testing set (should be 573)
print(test_len)

In [ ]:
#convert image paths into numpy matrices
def parse_image(filename):
  img_obj = Image.open(filename)
  img = np.asarray(img_obj).astype(np.float32)
  #normalize image to 0-1 range
  img /= 255.0
  return img

train_images = np.array(list(map(parse_image, train_images)))
test_images = np.array(list(map(parse_image, test_images)))

# Add 4th dimension to image arrays to allow for model.fit to take them as inputs
train_images = np.reshape(train_images, (-1, 227, 227, 1))
test_images = np.reshape(test_images, (-1, 227, 227, 1))

In [ ]:
# Ensure that training images have been created into a 3D array of 3247 images of 227x227 pixels
np.shape(train_images[0])

In [ ]:
print(train_images[0])

### Convert labels to one-hot encoding

In [ ]:
train_labels[0]

In [ ]:
def to_one_hot(label):
  encoding = [0 for _ in range(len(class_labels))]
  encoding[label] = 1
  return np.array(encoding).astype(np.float32)

train_labels = np.array(list(map(to_one_hot, train_labels)))
test_labels = np.array(list(map(to_one_hot, test_labels)))

train_labels = np.reshape(train_labels, (-1, 7))
test_labels = np.reshape(test_labels, (-1, 7))

In [ ]:
# Print one-hot encoding of labels (train_image[0] is classified as "dark_dune")
train_labels[0]

## 2. Define and Train the CNN - WIP

#### Code Source: https://github.com/niehusst/HiRISE-Net

This project by **niehusst** on GitHub sought to emulate the HiRISENet model used in the "Deep Mars" paper. The CNN defined in his project provided us a base CNN to use which we tuned to improve the test set accuracy.

In [ ]:
# make a generator to train the model with
generator = ImageDataGenerator(rotation_range=0, zoom_range=0,
    width_shift_range=0, height_shift_range=0, shear_range=0,
    horizontal_flip=False, fill_mode="nearest")

In [ ]:
###             BUILD SHAPE OF THE MODEL              ###

# increase kernel size and stride??
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
      input_shape=(227,227,1)),
  tf.keras.layers.MaxPooling2D((2,2), strides=2),
  tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
  tf.keras.layers.MaxPooling2D((2,2), strides=2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(7, activation=tf.nn.softmax), # final layer with node for each classification
])

# specify loss and SGD functions
model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
model.summary()

In [1]:
###                 TRAIN THE MODEL                   ###

#specify training metadata
BATCH_SIZE = 32

# train the model on the training data
num_epochs = 11
model.fit(generator.flow(train_images, train_labels, batch_size=BATCH_SIZE), epochs=num_epochs)

NameError: name 'model' is not defined

## 3. Define and Train the Vision Transformer (ViT) - WIP
Website used as a source: https://theaisummer.com/hugging-face-vit/

Original code that required Linux for JAX and Flax: https://github.com/google-research/vision_transformer/blob/main/vit_jax_augreg.ipynb

In [ ]:
from hugsvision.dataio.VisionDataset import VisionDataset

train, test, id2label, label2id = VisionDataset.fromImageFolder(
    "./hirise-map-proj-v3/data/",
    test_ratio   = 0.15,
    balanced     = False,
    augmentation = True,
);

In [ ]:
len(test)

In [ ]:
huggingface_model = 'google/vit-base-patch16-224-in21k'

In [ ]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification

model_name = "HIRISE_20EPOCH"

trainer = VisionClassifierTrainer(
    model_name   = model_name,
    train        = train,
    test         = test,
    output_dir   = "hirise-map-proj-v3/out/",
    max_epochs   = 5,
    batch_size   = 12, # On RTX 2080 Ti
    lr           = 2e-5,
    model = ViTForImageClassification.from_pretrained(
        huggingface_model,
        num_labels = len(label2id),
        label2id   = label2id,
        id2label   = id2label
    ),
    feature_extractor = ViTFeatureExtractor.from_pretrained(
        huggingface_model,
    ),
)

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ref, hyp)
labels = list(label2id.keys())
df_cm = pd.DataFrame(cm, index = labels, columns = labels)

plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, annot_kws={"size": 8}, fmt="")
plt.savefig("./hirise-map-proj-v3/out/"+model_name+"/conf_matrix_1.jpg")

print("Confusion Matrix saved to ./hirise-map-proj-v3/out/"+model_name+"/conf_matrix_1.jpg")

## 4. Evaluate CNN vs ViT - WIP

In [ ]:
# Evaluating the CNN - BEST IS 81% AFTER 5 EPOCHS
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Final loss was {}.\nAccuracy of model was {}".format(test_loss, test_accuracy))

In [ ]:
# Evaluate the ViT
ref, hyp = trainer.evaluate_f1_score()

##### ViT

Produces an accuracy of 70.83% when pre-trained on "vit-base-patch16-224-in21k" at 5 epochs (map-proj and map-proj-v3)

Produces an accuracy of 95.83% when pre-trained on "vit-base-patch16-224-in21k" at 20 epochs (map-proj and map-proj-v3)

In [ ]:
# Make a plot for the clout


#### Short response to our findings:
Was the output expected? what did we do for optimizations? is it overfit/underfit?

## 5. Retrain CNN and ViT on small, medium, and full HiRISE - WIP

In [ ]:
# Retrain 3 times each


## 6. Compare three CNNs vs three ViTs - WIP

### Print out a classification report to tabulate performance of the CNN

In [ ]:
## Generate predictions for the test data using the CNN:
cnn_label_pred = np.argmax(model.predict(test_images), axis=-1)
# convert to one hot encoding to compare to the test labels
cnn_label_pred = np.array(list(map(to_one_hot, cnn_label_pred)))
# print a classification report to observe performance of the CNN
import warnings
warnings.filterwarnings('ignore') # This was done because there was a warning with the f1-score that was breaking anything but didn't look presentable
print(classification_report(test_labels,cnn_label_pred))
cnn_report = classification_report(test_labels,cnn_label_pred, output_dict=True, labels=np.unique(test_labels))
import pandas as pd
cnn_report_df = pd.DataFrame(cnn_report).transpose()
cnn_report_df


In [ ]:
# Save the classification report as a csv
cnn_report_df.to_csv('cnn_classification_report.csv')

In [ ]:
# Plot for the clout


## Helper Code - Should not be run

In [ ]:
# Converting Data from map-proj-v3 : We need 7 folders named
# for each class label with all of the images that apply to that class

# import os
# import os.path
# from shutil import copy2

# from tqdm import tqdm

# import pandas as pd

# # label translations
# class_labels = ['other','crater','dark_dune','streak',
#                 'bright_dune','impact','swiss_cheese','spider']

# img_in = "C:/Users/Andrew/Documents/Final-Project---ECE-697ML/hirise-map-proj-v3/map-proj/"
# img_out = "C:/Users/Andrew/Documents/Final-Project---ECE-697ML/hirise-map-proj-v/data/"

# with open('hirise-map-proj-v3/labels-map-proj.txt') as labels:
#   for line in labels:
#     file_name, label = line.split(' ')
    
#     # Change label number into class label
#     class_temp = class_labels[int(label)]
    
#     # Update path to copy the image to
#     path_out_full = img_out + class_temp
    
#     # Get the full path of the image
#     path_in_full = img_in + file_name
    
#     # Check if the input image exist
#     if not os.path.isfile(path_in_full):
#         continue
        
#     # Check if the output dir of the label exist
#     if not os.path.isdir(path_out_full):
#         os.mkdir(path_out_full)

#     # Copy the image
#     copy2(path_in_full, path_out_full)

In [ ]:
# Converting Data from map-proj : We need 8 folders named
# for each class label with all of the images that apply to that class

# import os
# import os.path
# from shutil import copy2

# from tqdm import tqdm

# import pandas as pd

# # label translations
# class_labels = ['other','crater','dark_dune','streak',
#                 'bright_dune','impact','swiss_cheese','spider']

# img_in = "C:/Users/Andrew/Documents/Final-Project---ECE-697ML/hirise-map-proj-v3/map-proj-v3/"
# img_out = "C:/Users/Andrew/Documents/Final-Project---ECE-697ML/hirise-map-proj-v3/data/"

# with open('hirise-map-proj-v3/labels-map-proj-v3.txt') as labels:
#   for line in labels:
#     file_name, label = line.split(' ')
    
#     # Change label number into class label
#     class_temp = class_labels[int(label)]
    
#     # Update path to copy the image to
#     path_out_full = img_out + class_temp
    
#     # Get the full path of the image
#     path_in_full = img_in + file_name
    
#     # Check if the input image exist
#     if not os.path.isfile(path_in_full):
#         continue
        
#     # Check if the output dir of the label exist
#     if not os.path.isdir(path_out_full):
#         os.mkdir(path_out_full)

#     # Copy the image
#     copy2(path_in_full, path_out_full)